In [15]:
from langchain.agents import create_agent #for creating the agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit #for SQL database toolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate #for creating chat prompts templates
from langchain_google_genai import ChatGoogleGenerativeAI #for using Google Gemini LLM API
from dotenv import load_dotenv
import os

load_dotenv('../credentials.env') #load environment variables (credentials and API keys) from a .env file


True

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GOOGLE_API_KEY")
    )